In [15]:
# Install Deep500
! pip install git+https://github.com/deep500/deep500.git

  Cloning https://github.com/deep500/deep500.git to /tmp/pip-req-build-ytntplqd
  Stored in directory: /tmp/pip-ephem-wheel-cache-sfsry7ti/wheels/e3/8d/95/a4c60f0d7b00827c3a14476a62ba5c961876c158841e47cc36
Successfully built deep500


In [3]:
# Import Deep500
import deep500 as d5
import deep500.datasets as d5ds
import deep500.networks as d5nt

In [5]:
# Get dataset size
mnist_shape = d5ds.dataset_shape('mnist')
print(mnist_shape)
classes, c, h, w = mnist_shape
BATCH_SIZE = 128

(10, 1, 28, 28)


In [6]:
# Create deep neural network
onnx_file = d5nt.export_network('simple_cnn', BATCH_SIZE, classes=classes,
                                shape=(c, h, w))
model = d5.parser.load_and_parse_model(onnx_file)

/home/xl/d500/lib/python3.6/site-packages/deep500/networks/pytorch_mnist.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight, gain=np.sqrt(2))
/home/xl/d500/lib/python3.6/site-packages/deep500/networks/pytorch_mnist.py:12: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


graph(%0 : Float(128, 1, 28, 28)
      %1 : Float(10, 1, 5, 5)
      %2 : Float(10)
      %3 : Float(20, 10, 5, 5)
      %4 : Float(20)
      %5 : Float(50, 320)
      %6 : Float(50)
      %7 : Float(10, 50)
      %8 : Float(10)) {
  %9 : Float(128, 10, 24, 24) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%0, %1, %2), scope: Net/Conv2d[conv1]
  %10 : Float(128, 10, 12, 12) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%9), scope: Net
  %11 : Float(128, 10, 12, 12) = onnx::Relu(%10), scope: Net
  %12 : Float(128, 20, 8, 8) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%11, %3, %4), scope: Net/Conv2d[conv2]
  %13 : Float(128, 20, 4, 4) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%12), scope: Net
  %14 : Float(128, 20, 4, 4) = onnx::Relu(%13), scope: Net
  %15 : Tensor = onnx::Constant[value=  -1  320 [ CPULongType{2} ]](), scope: Net
 

In [8]:
# Get model input/output
INPUT_NODE = model.get_input_nodes()[0].name
OUTPUT_NODE = model.get_output_nodes()[0].name
print(INPUT_NODE, OUTPUT_NODE)

0 20


In [9]:
# Set up dataset
train_set, test_set = d5ds.load_dataset('mnist', INPUT_NODE, 'labels')
model.add_operation(d5.ops.SoftmaxCrossEntropy([OUTPUT_NODE, 'labels'], 'loss'))

0.00B [00:00, ?B/s]

/tmp/mnist/train-images-idx3-ubyte.gz


9.46MB [00:02, 4.94MB/s]                            
0.00B [00:00, ?B/s]

/tmp/mnist/t10k-images-idx3-ubyte.gz


1.58MB [00:00, 1.75MB/s]                            
 28%|██▊       | 8.00k/28.2k [00:00<00:00, 43.0kB/s]

/tmp/mnist/train-labels-idx1-ubyte.gz


32.0kB [00:00, 116kB/s]                             
8.00kB [00:00, 43.7kB/s]


/tmp/mnist/t10k-labels-idx1-ubyte.gz
Download complete.


12

In [10]:
# Set dataset samplers
train_sampler = d5.ShuffleSampler(train_set, BATCH_SIZE)
test_sampler = d5.ShuffleSampler(test_set, BATCH_SIZE)

In [11]:
# Set up network executor
from deep500.frameworks import tensorflow as d5fw
executor = d5fw.from_model(model)

In [12]:
# Set up opimizer
from deep500.frameworks import reference as d5ref
optimizer = d5ref.GradientDescent(executor, 'loss', 0.1)

In [13]:
EPOCHS = 2
d5.test_training(executor, train_sampler, test_sampler, optimizer, 
                 EPOCHS, BATCH_SIZE, OUTPUT_NODE)

Testing: 100%|██████████| 78/78 [00:01<00:00, 59.61it/s, accuracy=97.8, test_loss=0.0669]

TrainingAccuracy: 96.64963942307692
TestAccuracy: 97.83653846153847


[96.64963942307692, 97.83653846153847]

In [14]:
import numpy as np

class StochasticUpdateRule(d5ref.GradientDescent):
    def update_rule(self, grad, old_param, param_name):
        return old_param - (np.random.random_sample() * self.lr) * grad

# Reload model
model = d5.parser.load_and_parse_model(onnx_file)
model.add_operation(d5.ops.SoftmaxCrossEntropy([OUTPUT_NODE, 'labels'], 'loss'))
executor = d5fw.from_model(model) 

# Replace optimizer
optimizer = StochasticUpdateRule(executor, 'loss', 0.5)
d5.test_training(executor, train_sampler, test_sampler, optimizer, 
                 EPOCHS, BATCH_SIZE, OUTPUT_NODE)


Testing: 100%|██████████| 78/78 [00:01<00:00, 57.12it/s, accuracy=98.1, test_loss=0.0559]

TrainingAccuracy: 97.54607371794872
TestAccuracy: 98.08693910256412


[97.54607371794872, 98.08693910256412]